# P1: Trend Filters on BollingerMeanReversion — BTC/USDT

**Research question (from roadmap P1):**
> Trend filters *hurt* BollingerBreakout because they lag momentum entries.
> Does the same logic *help* BollingerMeanReversion, where the correct
> regime is a ranging/flat market?

**Filter logic (inverted from Breakout):**

| Variant | Logic |
|---|---|
| Unfiltered | trade all signals |
| ADX filter | only trade when `adx < threshold` (ranging, not trending) |
| Slope flat | only trade when slope is flat (`slope_dir == 0`) |
| Slope aligned | keep long when slope ≥ 0, keep short when slope ≤ 0 (trade with the bias) |
| Both (ADX + aligned) | ADX ranging AND slope-aligned |

**Expected outcome:** filters should improve mean reversion by avoiding
strong trending regimes where mean reversion repeatedly fails.

In [ ]:
# ── Configuration ─────────────────────────────────────────────────────────────
SYMBOL    = "BTC/USDT"
SINCE     = "2024-01-01"
UNTIL     = "2024-06-01"

BB_PERIOD  = 20
BB_NUM_STD = 2.0

ADX_PERIOD    = 14
ADX_THRESHOLD = 25.0   # sit OUT when adx >= this

MA_PERIOD      = 20
SLOPE_WINDOW   = 5
FLAT_THRESHOLD = 0.05

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

sys.path.insert(0, str(Path().resolve().parent))

from data.fetch import fetch_ohlcv
from signals.trend.adx import ADXTrend
from signals.trend.ma_slope import MASlopeTrend
from strategies.single.basic.bollinger_bands import BollingerMeanReversion
from backtesting.metrics import compute_metrics

In [ ]:
df = fetch_ohlcv(symbol=SYMBOL, since=SINCE, until=UNTIL)
print(f"Fetched {len(df):,} bars  |  {df.index[0]}  →  {df.index[-1]}")

## 1. Build signals and filtered variants

In [ ]:
# ── Compute signals ───────────────────────────────────────────────────────────
adx_df   = ADXTrend(period=ADX_PERIOD, trend_threshold=ADX_THRESHOLD).compute(df)
slope_df = MASlopeTrend(ma_period=MA_PERIOD, slope_window=SLOPE_WINDOW,
                        flat_threshold=FLAT_THRESHOLD).compute(df)

mr_df    = BollingerMeanReversion(period=BB_PERIOD, num_std=BB_NUM_STD).generate_signals(df)
base_sig = mr_df["signal"]

adx_ranging  = adx_df["adx"] < ADX_THRESHOLD          # ranging market (not trending)
slope_dir    = slope_df["trend_dir"]
slope_flat   = slope_dir == 0                          # slope is flat
slope_aligned = (base_sig == slope_dir) | slope_flat   # signal direction consistent with slope

# ── Filtered variants ─────────────────────────────────────────────────────────
adx_filtered    = base_sig.where(adx_ranging, 0)
flat_filtered   = base_sig.where(slope_flat, 0)
aligned_filtered = base_sig.where(slope_aligned, 0)
both_filtered   = base_sig.where(adx_ranging & slope_aligned, 0)

variants = {
    "Unfiltered":     base_sig,
    "ADX ranging":    adx_filtered,
    "Slope flat":     flat_filtered,
    "Slope aligned":  aligned_filtered,
    "ADX + aligned":  both_filtered,
}

print("Active bars per variant:")
for name, sig in variants.items():
    pct = (sig != 0).mean() * 100
    print(f"  {name:<18} {(sig != 0).sum():>5} bars  ({pct:.1f}%)")

## 2. Equity curves

In [ ]:
market_ret = df["close"].pct_change().fillna(0)

def build_equity(sig: pd.Series) -> pd.Series:
    return (1 + sig.shift(1).fillna(0) * market_ret).cumprod()

equities = {name: build_equity(sig) for name, sig in variants.items()}

colors = {
    "Unfiltered":    "grey",
    "ADX ranging":   "steelblue",
    "Slope flat":    "darkorange",
    "Slope aligned": "mediumorchid",
    "ADX + aligned": "green",
}

fig, ax = plt.subplots(figsize=(14, 5))
for name, equity in equities.items():
    ax.plot(equity.index, equity, label=name, color=colors[name],
            lw=1.8 if name != "Unfiltered" else 1.0,
            ls="--" if name == "Unfiltered" else "-")
ax.axhline(1, color="black", lw=0.5, ls=":", label="Baseline (1.0)")
ax.set_title(f"BollingerMeanReversion: filtered vs unfiltered  |  {SYMBOL}  {SINCE} → {UNTIL}")
ax.set_ylabel("Equity (normalised)")
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
ax.legend()
plt.tight_layout()
plt.show()

## 3. Metrics comparison

In [ ]:
all_metrics = {name: compute_metrics(eq) for name, eq in equities.items()}
comparison  = pd.DataFrame(all_metrics)

pct_rows = {"total_return", "mean_return", "std_return",
            "mean_neg_return", "std_neg_return",
            "return_p05", "return_p25", "return_p75", "return_p95",
            "max_drawdown", "win_rate"}

def fmt(row_name, val):
    if pd.isna(val): return "nan"
    return f"{val*100:.2f}%" if row_name in pct_rows else f"{val:.4f}"

formatted = comparison.apply(lambda col: [fmt(idx, v) for idx, v in col.items()])
formatted.index = comparison.index
formatted

In [ ]:
key_metrics = ["total_return", "sharpe_ratio", "sortino_ratio",
               "max_drawdown", "win_rate", "calmar_ratio"]
labels      = list(variants.keys())
bar_colors  = [colors[l] for l in labels]

fig, axes = plt.subplots(2, 3, figsize=(14, 7))
fig.suptitle("Key metrics: filtered vs unfiltered (BollingerMeanReversion)", fontsize=13)

for ax, metric in zip(axes.flat, key_metrics):
    vals = [all_metrics[l][metric] for l in labels]
    bars = ax.bar(labels, vals, color=bar_colors, alpha=0.8, edgecolor="black", lw=0.5)
    ax.axhline(0, color="black", lw=0.5)
    ax.set_title(metric)
    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=20, ha="right", fontsize=7)
    unfiltered_val = all_metrics["Unfiltered"][metric]
    if not np.isnan(unfiltered_val):
        ax.axhline(unfiltered_val, color="grey", lw=1, ls="--", alpha=0.7)
    for bar, val in zip(bars, vals):
        if not np.isnan(val):
            ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
                    f"{val*100:.1f}%" if metric in pct_rows else f"{val:.2f}",
                    ha="center", va="bottom", fontsize=7)

plt.tight_layout()
plt.show()

## 4. Regime visualisation — when do filters sit out?

Showing which bars each filter removes, overlaid on price, to understand
whether the sit-out periods align with visually trending regimes.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
fig.suptitle(f"Regime sit-out periods  |  {SYMBOL}  {SINCE} → {UNTIL}", fontsize=13)

price = df["close"]
lo, hi = price.min(), price.max()

# ADX ranging filter
ax = axes[0]
ax.plot(price.index, price, lw=0.8, color="black")
ax.fill_between(price.index, lo, hi, where=adx_ranging,
                alpha=0.15, color="steelblue", label="Trading (ADX < 25)")
ax.fill_between(price.index, lo, hi, where=~adx_ranging,
                alpha=0.15, color="tomato", label="Sit out (ADX ≥ 25, trending)")
ax.set_ylabel("Price")
ax.set_title("ADX ranging filter")
ax.legend(loc="upper left", fontsize=8)

# Slope flat filter
ax = axes[1]
ax.plot(price.index, price, lw=0.8, color="black")
ax.fill_between(price.index, lo, hi, where=slope_flat,
                alpha=0.15, color="darkorange", label="Trading (slope flat)")
ax.fill_between(price.index, lo, hi, where=~slope_flat,
                alpha=0.15, color="tomato", label="Sit out (slope trending)")
ax.set_ylabel("Price")
ax.set_title("Slope flat filter")
ax.legend(loc="upper left", fontsize=8)

# Combined filter
ax = axes[2]
trade_zone = adx_ranging & slope_aligned
ax.plot(price.index, price, lw=0.8, color="black")
ax.fill_between(price.index, lo, hi, where=trade_zone,
                alpha=0.15, color="green", label="Trading (ADX + aligned)")
ax.fill_between(price.index, lo, hi, where=~trade_zone,
                alpha=0.15, color="tomato", label="Sit out")
ax.set_ylabel("Price")
ax.set_title("Combined filter (ADX + slope aligned)")
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
ax.legend(loc="upper left", fontsize=8)

plt.tight_layout()
plt.show()

## 5. Interpretation & conclusions

In [ ]:
# Summary: did each filter improve over unfiltered?
base = all_metrics["Unfiltered"]
summary_rows = []
for name in ["ADX ranging", "Slope flat", "Slope aligned", "ADX + aligned"]:
    m = all_metrics[name]
    summary_rows.append({
        "variant":         name,
        "total_return":    f"{m['total_return']*100:.2f}%",
        "vs unfiltered":   f"{(m['total_return'] - base['total_return'])*100:+.2f}pp",
        "sharpe":          f"{m['sharpe_ratio']:.3f}",
        "sharpe_delta":    f"{m['sharpe_ratio'] - base['sharpe_ratio']:+.3f}",
        "max_drawdown":    f"{m['max_drawdown']*100:.2f}%",
        "dd_delta":        f"{(m['max_drawdown'] - base['max_drawdown'])*100:+.2f}pp",
        "coverage":        f"{(variants[name] != 0).mean()*100:.1f}%",
    })

summary = pd.DataFrame(summary_rows).set_index("variant")
display(summary)

### Key takeaways

_(Fill in after running — compare delta columns to assess whether hypothesis holds)_

**Hypothesis:** trend filters should improve mean reversion by avoiding strong trends.

**Questions to answer from the results:**
1. Do any filters improve Sharpe over unfiltered? By how much?
2. Does the drawdown reduce with filtering (less exposure to trending losses)?
3. Is the ADX or slope filter more useful for mean reversion?
4. Does tighter coverage (sitting out more) help or hurt?
5. Compare to the Breakout filter experiment — which strategy benefited more from filtering?

In [ ]:
# Side-by-side: MeanReversion vs Breakout filter impact
# (Shows whether filtering is more useful for one strategy type)
from strategies.single.basic.bollinger_bands import BollingerBreakout

bo_df    = BollingerBreakout(period=BB_PERIOD, num_std=BB_NUM_STD).generate_signals(df)
bo_base  = bo_df["signal"]
bo_adx   = bo_base.where(adx_df["adx"] >= ADX_THRESHOLD, 0)  # breakout: trade when trending

mr_unfiltered_eq = build_equity(base_sig)
mr_adx_eq        = build_equity(adx_filtered)
bo_unfiltered_eq = build_equity(bo_base)
bo_adx_eq        = build_equity(bo_adx)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle("Filter impact: MeanReversion vs Breakout  |  ADX filter only", fontsize=12)

for ax, unf, flt, title, c_unf, c_flt in [
    (axes[0], mr_unfiltered_eq, mr_adx_eq, "BollingerMeanReversion", "grey", "steelblue"),
    (axes[1], bo_unfiltered_eq, bo_adx_eq, "BollingerBreakout",      "grey", "tomato"),
]:
    ax.plot(unf.index, unf, color=c_unf, lw=1.0, ls="--", label="Unfiltered")
    ax.plot(flt.index, flt, color=c_flt, lw=1.5, label="ADX filtered")
    ax.axhline(1, color="black", lw=0.5, ls=":")
    ax.set_title(title)
    ax.set_ylabel("Equity (normalised)")
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
    ax.legend(fontsize=9)

plt.tight_layout()
plt.show()